<a href="https://colab.research.google.com/github/jtnishi/Jupyter_Relative_Currency_Performance/blob/main/Relative_Currency_Performance_Information_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relative Currency Performance Information Generator

**2022-10-22**

## Introduction

With the US Dollar reaching an exchange rate of ¥150 to $1, I wanted a way to be able to figure out what is the relative performance of these currencies against each other, to try to get a sense of how much of the exchange rate change is due to the strengthening of one currency relative to others, and how much is due to the weakening of another currency relative to others.  In my 
[original Google spreadsheet](https://docs.google.com/spreadsheets/d/11huoUDaFcvoWdmGiiSBFhur1t821qRrtlLUv9gt5xpY/edit?usp=sharing) for this idea, I used the Swiss Franc as the reference currency.  
  
This spreadsheet works, but it is kind of inelegant in how it looks and functions. I'd rather have some code do the work to output the data and give me graphs.  Ergo, let's do this with some Python.  
  
---


## Methodology

I will start with a reference date and a reference currency.  That will effectively act as a time zero.  We will calculate the conversion of each currency compared to the reference currency.  We will take that as the 100% point of the data.  We can then take the conversion of the currency from each day going forward, and calculate it relative to the original conversion.  That will give us a relative performance of each currency compared to the reference.

Ideally, we want to pick a reference currency that kind of reflects something like a "world average".  Google Finance doesn't support the IMF's Special Drawing Rights (XDR).  So I originally picked the Swiss Franc (CHF) as a currency that should be neutral-ish and not super involved in the politics of the world, due to Switzerland being mostly neutral famously.  
  
With this code, I can actually use any API that I can access with Python code, so I may instead choose to use a different data source.  

For this case, we'll use [this api](https://github.com/fawazahmed0/currency-api#readme) as our base, because it's a simple one that just basically just pulls the data out of a CDN.  This works just fine.

---

---
---
---

## Execution

For those of you that haven't dealt with Jupyter Notebooks or Google Colab, this is what you need to do:

1. Upper right hand corner, click on the triangle menu, and click "Connect to a hosted runtime."
2. For each section, follow the instructions in each section above if any to make modifications, and then hit the play button next to each chunk of Python code.
3. Go one step at a time, waiting until each segment finishes. You'll see a checkmark after each one.  For example, the next piece of code, if it works should just output:
```
    Congratulations, you've run this successfully.
```

In [1]:
print("Congratulations, you've run this successfully")

Congratulations, you've run this successfully


---

### Step 0 - RUN THIS FIRST

Here, we're going to define some basic parts of the Python code.  This will cover imports, some basic utility functions, and some constant definitons.

In [2]:
##########################
#  === PIP INSTALLS ===  #
##########################

# For graph stuff.
!pip install --upgrade plotly

#####################
#  === IMPORTS ===  #
#####################

from datetime import date as DTDate, timedelta as DTDelta
import json
import logging
import pprint

import pandas as pd
import requests
import tabulate

#######################
#  === CONSTANTS ===  #
#######################

# https://github.com/fawazahmed0/currency-api#readme
API_URL_FORMAT = 'https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@{apiVersion}/{date}{endpoint}.json'

# Per the documentation, the API version hasn't really changed.
API_VERSION = 1

####################
#  === LOGGER ===  #
####################

logging.basicConfig(level=logging.INFO, force=True)
#logging.basicConfig(level=logging.DEBUG, force=True)
LOG = logging.getLogger('log')
LOG.setLevel(logging.INFO)
#LOG.setLevel(logging.DEBUG)

################################################################################
################################################################################
################################################################################


def log_obj(obj_to_log, title='Logged Object', log_func=LOG.debug):
  """
  Log the object sent to the function to the system.  Useful for checking the
  format of data.
  """

  log_func('%s:' % title)
  log_func('')
  for line in pprint.pformat(obj_to_log).split('\n'):
    log_func('  %s' % line)


def get_api_data(endpoint, date='latest', api_version=API_VERSION):
  """
  Get the data out of the API, given dates and endpoint.
  """
  if isinstance(date, DTDate):
    date = date.isoformat()

  data_url = API_URL_FORMAT.format(apiVersion=api_version, date=date, endpoint=endpoint)
  LOG.debug('API Data Call to: %s', data_url)

  try:
    response = requests.get(data_url)
    response.raise_for_status()
    response_body = response.json()
    LOG.debug('API Data Call to %s response code: %d', data_url, response.status_code)
    log_obj(response_body, 'Response to %s' % data_url, log_func=LOG.debug)
    return response_body

  except requests.HTTPError as exc:
    LOG.error("Error in retrieving data. Exception: %s" % exc)
    return None


def get_currencies():
  """
  Gets the list of currencies from the API.
  """
  return get_api_data('/currencies.min')


def currency_data_mapping(currencies):
  """
  Maps all currency names to the full name.
  """
  # Convert singleton currency to list.
  if not isinstance(currencies, list):
    currencies = [currencies]

  ret_data = {'BAD_CURRENCIES': []}
  valid_currencies = get_currencies()

  for currency in currencies:
    if currency in valid_currencies:
      ret_data[currency] = valid_currencies[currency]
    else:
      ret_data['BAD_CURRENCIES'].append(currency)
  
  return ret_data


def currencies_in_data(currencies):
  """
  Validates that each of the currencies are in the data set. Throws an
  exception if a currency is not in the set.
  """

  currency_map = currency_data_mapping(currencies)

  # Finally, return information and error as needed.
  if not currency_map['BAD_CURRENCIES']:
    LOG.debug('All currencies found: %s', currencies)
    del currency_map['BAD_CURRENCIES']
    return currency_map

  errmsg = 'Currencies not found in set: %s' % pprint.pformat(currency_map['BAD_CURRENCIES'])
  raise ValueError(errmsg)


def get_date_ranges(start_date, end_date=DTDate.today()):
  """
  Generates the list of dates we will need to use to get information out of
  the API.
  """
  num_days = (end_date - start_date).days + 1
  dates = [(start_date + DTDelta(days=i)).isoformat()
           for i in range(num_days)]
  return dates

  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 5.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


---

### *Optional - Reference - All Valid Currencies and their Symbols*

Before we start, just to make sure we have a tabular list of all available currencies to convert from, you can run the following cell to generate a table of all the currencies in the reference data list from the API.  This will be used in the configuration step below.

**NOTE: This includes a mish-mash of both cryptocurrencies AND regular currencies.  You will want to filter accordingly.**

In [5]:
def print_currency_table(data):
  if data:
    table_items = [[symbol, name] for symbol, name in data.items()]
    return tabulate.tabulate(table_items, headers=('Symbol', 'Currency Name'), tablefmt='simple')
  return None

data = get_currencies()
formatted = print_currency_table(data)

print('Table of currencies:\n')
print(formatted)

Table of currencies:

Symbol    Currency Name
--------  --------------------------------------
1inch     1inch Network
aave      Aave
ada       Cardano
aed       United Arab Emirates Dirham
afn       Afghan afghani
algo      Algorand
all       Albanian lek
amd       Armenian dram
amp       Synereo
ang       Netherlands Antillean Guilder
aoa       Angolan kwanza
ar        Arweave
ars       Argentine peso
atom      Atomic Coin
aud       Australian dollar
avax      Avalanche
awg       Aruban florin
axs       AXS
azn       Azerbaijani manat
bam       Bosnia-Herzegovina Convertible Mark
bat       Basic Attention Token
bbd       Bajan dollar
bch       Bitcoin Cash
bdt       Bangladeshi taka
bgn       Bulgarian lev
bhd       Bahraini dinar
bif       Burundian Franc
bmd       Bermudan dollar
bnb       Binance Coin
bnd       Brunei dollar
bob       Bolivian boliviano
brl       Brazilian real
bsd       Bahamian dollar
bsv       Bitcoin SV
btc       Bitcoin
btcb      Bitcoin BEP2
btg       Bitcoi

---

### Step 1. Configuration.

Use the next box to configure the parameters to be used by the executing code. This will be used to make the right calls to the finance APIs.  You can fill in the form as needed.

In [4]:
###########################
#  === CONFIGURATION ===  #
###########################

# Put the symbol of the reference currency that all other currencies will be
# compared against.  For the table of all possible symbols, use the data from
# the cell above.
#REFERENCE_CURRENCY = 'chf'  # Swiss Franc
REFERENCE_CURRENCY = 'xdr'  # IMF Special Drawing Rights

# Create the list of currencies to convert relative to the reference currency.
CONVERSION_CURRENCIES = [
    'usd',  # United States dollar
    'jpy',  # Japanese Yen
    'eur',  # Euro
    'gbp',  # Pound Sterling
    'cny',  # Chinese Yuan
    'inr',  # Indian Rupee
    'cad',  # Canadian Dollar
    'aud',  # Australian Dollar
    'mxn',  # Mexican Peso
    'chf',  # Swiss Franc
    'sek',  # Swedish Krona
    'nzd',  # New Zealand Dollar
    'rub',  # Russian Ruble
    'hkd',  # Hong Kong Dollar
    'zar',  # South African Rand
    'xdr',  # IMF Special Drawing Rights

    # Some cryptos mostly for (making) fun (of).
    'btc',  # Bitcoin
    'eth',  # Ethereum
    'doge',  # Dogecoin
]

START_DATE = DTDate.fromisoformat('2022-01-01')
END_DATE = DTDate.today()


#### *Optional - Data Sanity Check*

The following cell runs a sanity check on the configuration data above and logs the information to the notebook.

In [6]:
# Sanity check of data

def step_01_config_sanity_check(currency_list, start_date, end_date):
  """
  Runs a quick sanity check of the data to make sure that
  the config looks sound.
  """
  mapping = currencies_in_data(currency_list)
  log_obj(mapping, "Currency Mapping", log_func=LOG.info)
  dates = get_date_ranges(start_date, end_date)
  LOG.info('Number of dates: %d', len(dates))
  LOG.info('First Date: %s', dates[0])
  LOG.info('Last Date: %s', dates[-1])
  log_obj(dates, "Dates in Range", log_func=LOG.debug)

step_01_config_sanity_check(
    [REFERENCE_CURRENCY] + CONVERSION_CURRENCIES,
    START_DATE,
    END_DATE
)

INFO:log:Currency Mapping:
INFO:log:
INFO:log:  {'aud': 'Australian dollar',
INFO:log:   'btc': 'Bitcoin',
INFO:log:   'cad': 'Canadian dollar',
INFO:log:   'chf': 'Swiss franc',
INFO:log:   'cny': 'Chinese Yuan',
INFO:log:   'doge': 'Dogecoin',
INFO:log:   'eth': 'Ether',
INFO:log:   'eur': 'Euro',
INFO:log:   'gbp': 'Pound sterling',
INFO:log:   'hkd': 'Hong Kong dollar',
INFO:log:   'inr': 'Indian rupee',
INFO:log:   'jpy': 'Japanese yen',
INFO:log:   'mxn': 'Mexican peso',
INFO:log:   'nzd': 'New Zealand dollar',
INFO:log:   'rub': 'Russian ruble',
INFO:log:   'sek': 'Swedish krona',
INFO:log:   'usd': 'United States dollar',
INFO:log:   'xdr': 'Special Drawing Rights',
INFO:log:   'zar': 'South African rand'}
INFO:log:Number of dates: 294
INFO:log:First Date: 2022-01-01
INFO:log:Last Date: 2022-10-21


---

### Step 2: Run through the API and gather data

This does the hard work step of going through the API, gathering up the data from the individual date records, and compiling them into a couple of data frames that will represent the actual financial data.

In [7]:
def data_for_date(reference_currency, date, conversion_currencies):
  """
  For a given date and reference currency, get the conversion data for the
  currencies.
  """
  if not isinstance(conversion_currencies, list):
    conversion_currencies = [conversion_currencies]

  endpoint = f'/currencies/{reference_currency}'
  data = get_api_data(endpoint, date)

  if not data:
    data = {}

  data_date = data.get('date')
  LOG.debug('Captured date: %s', data_date)

  if reference_currency in data:
    data = data[reference_currency]
  else:
    data = {}

  if not data:
    LOG.warning('Data missing for reference currency %s for date %s',
                reference_currency,
                date)
    return {currency: None for currency in conversion_currencies}

  ret_data = {}
  for currency in conversion_currencies:
    if currency in data:
      # NOTE: When we do the power relationship, we need to use the INVERSE
      # of each of the currencies.  That is, convert 1 of the base currency to
      # our reference.  For now, we'll just store the raw data.  That's useful
      # to refer to from the beginning
      ret_data[currency] = data[currency]

    else:
      LOG.warning('Data missing for currency %s in data for base currency %s '
                  'for date %s',
                  currency, reference_currency, date)
      ret_data[currency] = None

  return ret_data


def data_for_dates(reference_currency, start_date, end_date, conversion_currencies):
  """
  Get the currency conversion data for a full set of dates.
  """
  dates = get_date_ranges(start_date, end_date)
  
  # @TODO: Parallelize this!
  ret_data = {}
  len_dates = len(dates)

  LOG.info('Retrieving data for %d dates.  This will take a while.', len_dates)
  for idx, date in enumerate(dates):
    LOG.debug('Getting data for %s', date)
    ret_data[date] = data_for_date(reference_currency, date, conversion_currencies)

    cur_cnt = idx + 1
    if cur_cnt % 10 == 0:
      LOG.info('Finished %d of %d dates.', cur_cnt, len_dates)

  LOG.info('Done retrieving information for all dates.')
  ret_pd = pd.DataFrame.from_dict(ret_data)

  return ret_pd


def inverted_conversions(currency_data):
  """
  For graphing relative power of each currency, we actually need the inverse of
  the input.  That is, we need to convert 1 unit of the currencies we want as
  our lookups to our reference currency.  The base table gives us the opposite,
  one unit of the reference currency in the others.  However, doing the call
  this way does far fewer API calls for our API.
  """
  return currency_data.pow(-1)


def relative_currency_power(inverse_currency_data):
  """
  Finally, calculate the relative value of each currency to the reference
  currency on the initial date.
  """
  first_date = inverse_currency_data.columns[0]
  base_data = inverse_currency_data[first_date]
  relative_power_data = inverse_currency_data.div(base_data, axis=0)
  return relative_power_data


def remap_data(relative_data):
  """
  For final presentation, convert the labels of the base currencies to the
  full labels of the currencies.
  """
  currencies = list(relative_data.index)
  currency_map = currency_data_mapping(currencies)
  final_map = {
      currency: '%s (%s)' % (name, currency.upper())
      for currency, name in currency_map.items()
      if currency != 'BAD_CURRENCIES'
  }
  renamed_data = relative_data.rename(mapper=final_map, axis=0)
  return renamed_data

###############################################################################

def step_02_full_data_process(reference_currency, start_date, end_date, conversion_currencies):
  """
  Handle all the processing of step 2.
  """
  base_currency_data = data_for_dates(reference_currency, start_date, end_date, conversion_currencies)
  inverted_bc_data = inverted_conversions(base_currency_data)
  relative_bc_data = relative_currency_power(inverted_bc_data)
  renamed_data = remap_data(relative_bc_data)

  ret_data = {
      'data': renamed_data,      
      'label': 'Relative Currency Strength',
      'reference': {
        'Raw Currency Data': base_currency_data,
        'Inverse Conversion Data (X->Ref)': inverted_bc_data,
        'Relative Conversion Data': relative_bc_data,
      }
  }

  return ret_data

###############################################################################

step_02_data = step_02_full_data_process(REFERENCE_CURRENCY, START_DATE, END_DATE, CONVERSION_CURRENCIES)



INFO:log:Retrieving data for 294 dates.  This will take a while.
INFO:log:Finished 10 of 294 dates.
INFO:log:Finished 20 of 294 dates.
INFO:log:Finished 30 of 294 dates.
INFO:log:Finished 40 of 294 dates.
INFO:log:Finished 50 of 294 dates.
INFO:log:Finished 60 of 294 dates.
INFO:log:Finished 70 of 294 dates.
INFO:log:Finished 80 of 294 dates.
INFO:log:Finished 90 of 294 dates.
INFO:log:Finished 100 of 294 dates.
INFO:log:Finished 110 of 294 dates.
INFO:log:Finished 120 of 294 dates.
ERROR:log:Error in retrieving data. Exception: 403 Client Error: Forbidden for url: https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@1/2022-05-01/currencies/xdr.json
INFO:log:Finished 130 of 294 dates.
INFO:log:Finished 140 of 294 dates.
INFO:log:Finished 150 of 294 dates.
INFO:log:Finished 160 of 294 dates.
INFO:log:Finished 170 of 294 dates.
INFO:log:Finished 180 of 294 dates.
INFO:log:Finished 190 of 294 dates.
INFO:log:Finished 200 of 294 dates.
INFO:log:Finished 210 of 294 dates.
INFO:log:Finished 

#### *Optional - Data Table*

This shows the core data gathered after the retrieval phase

In [8]:
step_02_data['data']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
United States dollar (USD),1.0,1.000000,0.999999,1.000199,0.998442,1.002261,1.000009,1.000712,0.995739,0.995739,...,1.092520,1.096316,1.091802,1.096785,1.096785,1.095193,1.085981,1.095554,1.094371,1.095600
Japanese yen (JPY),1.0,1.000000,0.999058,0.998112,0.989609,0.993497,0.993416,0.996307,0.991604,0.991000,...,0.860607,0.859673,0.852493,0.848532,0.848532,0.848347,0.839953,0.845367,0.840535,0.839597
Euro (EUR),1.0,1.000000,0.999709,0.994490,0.990776,0.997150,0.993209,0.999853,0.994885,0.994049,...,0.931783,0.936501,0.939113,0.937826,0.937826,0.938093,0.939666,0.950110,0.939668,0.941524
Pound sterling (GBP),1.0,1.000856,1.000278,0.997037,0.999392,1.004978,1.000901,1.006269,1.000789,1.001016,...,0.885618,0.901184,0.913818,0.906274,0.906274,0.909545,0.912105,0.919102,0.907426,0.908333
Chinese Yuan (CNY),1.0,1.000000,1.000001,1.000169,0.998536,1.002244,0.995765,0.997292,0.992336,0.992337,...,0.969055,0.971188,0.967177,0.969410,0.969410,0.968009,0.959132,0.966834,0.962224,0.964893
Indian rupee (INR),1.0,1.000000,1.000000,1.001686,0.998159,1.003388,1.001295,1.001337,0.998910,0.998910,...,0.989480,0.993281,0.988336,0.991600,0.991600,0.987838,0.984343,0.992069,0.982488,0.985648
Canadian dollar (CAD),1.0,1.000000,0.999051,0.991532,0.993365,0.992808,0.993063,1.000633,0.995227,0.995035,...,1.000539,1.003426,1.005586,0.998670,0.998670,0.999387,1.000829,1.008829,1.004256,1.004914
Australian dollar (AUD),1.0,1.000000,1.000658,0.990546,0.994867,0.995781,0.985247,0.988604,0.983479,0.983561,...,0.940049,0.947971,0.950780,0.935351,0.935351,0.937518,0.941630,0.951687,0.942484,0.945126
Mexican peso (MXN),1.0,1.000000,1.000495,1.000630,0.996638,0.998395,0.999410,1.005910,1.001480,1.000588,...,1.115534,1.124257,1.119900,1.119872,1.119872,1.120458,1.113988,1.121779,1.114462,1.120295
Swiss franc (CHF),1.0,1.000000,0.999064,0.993535,0.993798,0.996006,0.989411,0.992886,0.988262,0.987737,...,0.998014,1.002886,0.996059,0.993643,0.993643,0.994490,0.994848,1.004652,0.992503,0.993945


#### *Optional - Raw Currency Data*

The raw currency data for the reference currency converted to each of the targeted currencies is below.

In [9]:
step_02_data['reference']['Raw Currency Data']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
usd,1.399282,1.399282,1.399283,1.399003,1.401466,1.396125,1.399270,1.398286,1.405270,1.405270,...,1.280784,1.276349,1.281626,1.275803,1.275803,1.277658,1.288496,1.277237,1.278618,1.277183
jpy,161.068709,161.068709,161.220505,161.373455,162.759977,162.123026,162.136207,161.665763,162.432433,162.531448,...,187.157048,187.360371,188.938514,189.820389,189.820389,189.861869,191.759226,190.531144,191.626390,191.840444
eur,1.230522,1.230522,1.230880,1.237340,1.241978,1.234039,1.238935,1.230703,1.236849,1.237889,...,1.320610,1.313957,1.310302,1.312100,1.312100,1.311727,1.309531,1.295137,1.309528,1.306947
gbp,1.034978,1.034093,1.034690,1.038054,1.035608,1.029851,1.034046,1.028530,1.034162,1.033928,...,1.168651,1.148465,1.132586,1.142015,1.142015,1.137907,1.134714,1.126075,1.140565,1.139426
cny,8.893707,8.893707,8.893702,8.892203,8.906745,8.873796,8.931528,8.917858,8.962397,8.962389,...,9.177710,9.157555,9.195531,9.174351,9.174351,9.187629,9.272663,9.198791,9.242870,9.217296
inr,104.266235,104.266235,104.266255,104.090727,104.458496,103.914138,104.131374,104.126981,104.379967,104.379960,...,105.374752,104.971491,105.496798,105.149514,105.149514,105.549882,105.924731,105.099783,106.124659,105.784469
cad,1.768574,1.768574,1.770254,1.783678,1.780387,1.781386,1.780928,1.767455,1.777055,1.777399,...,1.767622,1.762536,1.758749,1.770930,1.770930,1.769658,1.767109,1.753096,1.761078,1.759926
aud,1.925134,1.925134,1.923868,1.943507,1.935067,1.933290,1.953960,1.947326,1.957473,1.957310,...,2.047909,2.030794,2.024795,2.058194,2.058194,2.053436,2.044470,2.022864,2.042617,2.036908
mxn,28.681526,28.681526,28.667331,28.663477,28.778268,28.727629,28.698468,28.513028,28.639132,28.664677,...,25.711028,25.511540,25.610796,25.611426,25.611426,25.598048,25.746720,25.567889,25.735757,25.601759
chf,1.275732,1.275732,1.276927,1.284033,1.283694,1.280848,1.289385,1.284872,1.290885,1.291570,...,1.278271,1.272061,1.280779,1.283894,1.283894,1.282800,1.282338,1.269825,1.285368,1.283504


#### *Optional - Inverse Conversion Data*

The data for the inverse conversion (ie: converting 1 unit of each currency to our reference currency), done by literally taking 1/x for each cell.

In [10]:
step_02_data['reference']['Inverse Conversion Data (X->Ref)']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
usd,0.714652,0.714652,0.714652,0.714795,0.713539,0.716268,0.714658,0.715161,0.711607,0.711607,...,0.780772,0.783485,0.780259,0.783820,0.783820,0.782682,0.776099,0.782940,0.782094,0.782973
jpy,0.006209,0.006209,0.006203,0.006197,0.006144,0.006168,0.006168,0.006186,0.006156,0.006153,...,0.005343,0.005337,0.005293,0.005268,0.005268,0.005267,0.005215,0.005248,0.005218,0.005213
eur,0.812663,0.812663,0.812427,0.808185,0.805167,0.810347,0.807145,0.812544,0.808506,0.807827,...,0.757226,0.761060,0.763183,0.762137,0.762137,0.762354,0.763632,0.772119,0.763634,0.765142
gbp,0.966204,0.967031,0.966473,0.963341,0.965616,0.971014,0.967075,0.972261,0.966966,0.967185,...,0.855687,0.870727,0.882935,0.875645,0.875645,0.878806,0.881279,0.888040,0.876758,0.877635
cny,0.112439,0.112439,0.112439,0.112458,0.112274,0.112691,0.111963,0.112135,0.111577,0.111577,...,0.108960,0.109199,0.108748,0.109000,0.109000,0.108842,0.107844,0.108710,0.108192,0.108492
inr,0.009591,0.009591,0.009591,0.009607,0.009573,0.009623,0.009603,0.009604,0.009580,0.009580,...,0.009490,0.009526,0.009479,0.009510,0.009510,0.009474,0.009441,0.009515,0.009423,0.009453
cad,0.565427,0.565427,0.564891,0.560639,0.561676,0.561361,0.561505,0.565785,0.562729,0.562620,...,0.565732,0.567364,0.568586,0.564675,0.564675,0.565081,0.565896,0.570419,0.567834,0.568206
aud,0.519444,0.519444,0.519786,0.514534,0.516778,0.517253,0.511781,0.513525,0.510863,0.510905,...,0.488303,0.492418,0.493877,0.485863,0.485863,0.486989,0.489124,0.494349,0.489568,0.490940
mxn,0.034866,0.034866,0.034883,0.034888,0.034748,0.034810,0.034845,0.035072,0.034917,0.034886,...,0.038894,0.039198,0.039046,0.039045,0.039045,0.039065,0.038840,0.039112,0.038856,0.039060
chf,0.783864,0.783864,0.783130,0.778796,0.779002,0.780733,0.775564,0.778288,0.774662,0.774251,...,0.782307,0.786126,0.780775,0.778880,0.778880,0.779545,0.779826,0.787510,0.777987,0.779117


---

### Step 3: Visualize the Core data

Now that we have the data calculated in step 2 above, we can send the data to be visualized.

For this we'll use one of the multitude of graphing libraries appropriate for this.

In [11]:
# Starting Data
step_02_data['data']

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2022-10-12,2022-10-13,2022-10-14,2022-10-15,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21
United States dollar (USD),1.0,1.000000,0.999999,1.000199,0.998442,1.002261,1.000009,1.000712,0.995739,0.995739,...,1.092520,1.096316,1.091802,1.096785,1.096785,1.095193,1.085981,1.095554,1.094371,1.095600
Japanese yen (JPY),1.0,1.000000,0.999058,0.998112,0.989609,0.993497,0.993416,0.996307,0.991604,0.991000,...,0.860607,0.859673,0.852493,0.848532,0.848532,0.848347,0.839953,0.845367,0.840535,0.839597
Euro (EUR),1.0,1.000000,0.999709,0.994490,0.990776,0.997150,0.993209,0.999853,0.994885,0.994049,...,0.931783,0.936501,0.939113,0.937826,0.937826,0.938093,0.939666,0.950110,0.939668,0.941524
Pound sterling (GBP),1.0,1.000856,1.000278,0.997037,0.999392,1.004978,1.000901,1.006269,1.000789,1.001016,...,0.885618,0.901184,0.913818,0.906274,0.906274,0.909545,0.912105,0.919102,0.907426,0.908333
Chinese Yuan (CNY),1.0,1.000000,1.000001,1.000169,0.998536,1.002244,0.995765,0.997292,0.992336,0.992337,...,0.969055,0.971188,0.967177,0.969410,0.969410,0.968009,0.959132,0.966834,0.962224,0.964893
Indian rupee (INR),1.0,1.000000,1.000000,1.001686,0.998159,1.003388,1.001295,1.001337,0.998910,0.998910,...,0.989480,0.993281,0.988336,0.991600,0.991600,0.987838,0.984343,0.992069,0.982488,0.985648
Canadian dollar (CAD),1.0,1.000000,0.999051,0.991532,0.993365,0.992808,0.993063,1.000633,0.995227,0.995035,...,1.000539,1.003426,1.005586,0.998670,0.998670,0.999387,1.000829,1.008829,1.004256,1.004914
Australian dollar (AUD),1.0,1.000000,1.000658,0.990546,0.994867,0.995781,0.985247,0.988604,0.983479,0.983561,...,0.940049,0.947971,0.950780,0.935351,0.935351,0.937518,0.941630,0.951687,0.942484,0.945126
Mexican peso (MXN),1.0,1.000000,1.000495,1.000630,0.996638,0.998395,0.999410,1.005910,1.001480,1.000588,...,1.115534,1.124257,1.119900,1.119872,1.119872,1.120458,1.113988,1.121779,1.114462,1.120295
Swiss franc (CHF),1.0,1.000000,0.999064,0.993535,0.993798,0.996006,0.989411,0.992886,0.988262,0.987737,...,0.998014,1.002886,0.996059,0.993643,0.993643,0.994490,0.994848,1.004652,0.992503,0.993945


Let's turn the above data now into a line plot.

In [12]:
import plotly.express as px

def fix_data_table(data_table):
  """
  We need to clean up the data table a bit.

  This involves un-pivoting the data, and renaming to something useful for the graph.
  """
  # Convert the data into long format data
  modded_data = data_table.reset_index()
  melted = modded_data.melt(
      id_vars='index',
      var_name='date',
      value_name='value'
  )

  # And rename the columns
  output_column_names = {
    'index': 'Currency',
    'date': 'Date',
    'value': 'Relative Currency Value'
  }

  melted.rename(columns=output_column_names, inplace=True)
  return melted


def generate_title(ref_currency, start, end):
  """
  Get the title name for our graph.
  """
  return (f'Relative Value of Currencies compared to {ref_currency} from '
          f'{start} to {end}')


###############################################################################

def step_03_visualize(data, ref_currency, start, end):
  """
  Handle the actual visualization generation.
  """
  title = generate_title(ref_currency, start, end)
  fixed_data = fix_data_table(data)
  figure = px.line(
    fixed_data,
    x='Date',
    y='Relative Currency Value',
    color='Currency',
    title=title
  )
  figure.show()


###############################################################################

# And run!
step_03_visualize(step_02_data['data'], REFERENCE_CURRENCY, START_DATE, END_DATE)

## DONE!

Thanks for reading!